In [1]:
import torch
import torch.nn.functional as F

class VPLoss:
    def __call__(self, model, x_0, t, noise):
        x_t = self.q_sample(x_0, t, noise)
        pred_noise = model(x_t, t)
        return F.mse_loss(pred_noise, noise) # recheck the loss function

    def q_sample(self, x_0, t, noise):
        # Apply forward process to add noise
        # return x_t
        pass


In [ ]:
# def c_skip(sigma): return 1 / (sigma**2 + 1)**0.5
# def c_out(sigma): return sigma / (sigma**2 + 1)**0.5
# def c_in(sigma): return 1 / (sigma**2 + 1)**0.5
# def c_noise(sigma): return log(sigma)  # or some scaled version
